# Gravitational Wave Detection

This notebook runs the EfficientNet model for this project. This is adapted from the original work from this [kaggle notebook](https://www.kaggle.com/rijuvaish/gravitational-wave-detection-ensemble-tf/), similar to many other public ntoebooks from the competition.

In [1]:
# Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # plotting tools
from random import shuffle
import math
import os

# Train test split
from sklearn.model_selection import train_test_split

# Import tensorflow and keras
import tensorflow as tf
import keras

# Model & compile arguments
from tensorflow.keras.models import Sequential
from keras.utils import Sequence
from tensorflow.keras.optimizers import Adam

# Get the layers
from keras.layers import InputLayer
from keras.layers import Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense

# Import the Efficientnet models
!pip install -U -q efficientnet
import efficientnet.keras as efn

# TF model metrics
from tensorflow.keras.metrics import AUC

# 
import librosa
import torch

# (Install &) Import the nnAudio library for Constant Q-Transform
try:
    from nnAudio.Spectrogram import CQT1992v2
except:
    !pip install -q nnAudio
    from nnAudio.Spectrogram import CQT1992v2

     |████████████████████████████████| 50 kB 1.1 MB/s eta 0:00:01


In [24]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sample_sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [1]:
# function to return the npy file corresponding to the id
def get_npy_filepath(id_, is_train=True):
    path = ''
    if is_train:
        return f'../input/g2net-gravitational-wave-detection/train/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'
    else:
        return f'../input/g2net-gravitational-wave-detection/test/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'

In [2]:
# let's define some signal parameters
sample_rate = 2048 # data is provided at 2048 Hz
signal_length = 2 # each signal lasts 2 s
fmin, fmax = 20, 1024 # filter above 20 Hz, and max 1024 Hz (Nyquist freq = sample_rate/2)
hop_length = 64 # hop length parameter for the stft

In [10]:
# Define the Constant Q-Transform
cq_transform = CQT1992v2(sr=sample_rate, fmin=fmin, fmax=fmax, hop_length=hop_length)

# function to load the file, preprocess, return the respective Constant Q-transform
def parse_function(id_path, is_train):
    # load the npy file
    signals = np.load(get_npy_filepath(id_path, is_train))
    
    # loop through each signal
    for i in range(signals.shape[0]):
        # normalize the signal data
        signals[i] /= np.max(signals[i])
    
    # stack the arrays into a single vector
    signals = np.hstack(signals)
    
    # convert the signals to torch.tensor to pass to CQT
    signals = torch.from_numpy(signals).float()
    
    # get the CQT
    image = cq_transform(signals)
    
    # conver the image from torch.tensor to array
    image = np.array(image)
    
    # transpose the image to get right orientation
    image = np.transpose(image,(1,2,0))
    
    # conver the image to tf.tensor and return
    return image

CQT kernels created, time used = 0.0520 seconds


/opt/conda/lib/python3.7/site-packages/nnAudio/utils.py:326: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn('If fmax is given, n_bins will be ignored',SyntaxWarning)


In [11]:
class Dataset(Sequence):
    def __init__(self,idx,y=None,batch_size=256,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        if y is not None:
            self.is_train=True
        else:
            self.is_train=False
        self.y = y
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
            
        list_x = np.array([parse_function(x,self.is_train) for x in batch_ids])
        batch_X = np.stack(list_x)
        if self.is_train:
            return batch_X, batch_y
        else:
            return batch_X
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [12]:
# Get the feature ids and target
X = train[['id']]
y = train['target'].astype('int8').values

In [ ]:
# Split the training IDs into training & validation datasets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, stratify=y)

# Assign the test IDs
X_test = sample_sub[['id']]

In [14]:
train_dataset = Dataset(X_train['id'], y_train)
valid_dataset = Dataset(X_valid['id'], y_valid)
test_dataset = Dataset(X_test['id'])

In [15]:
train_dataset

In [16]:
input_shape = (69, 193, 1)

In [17]:
model = tf.keras.Sequential([InputLayer(input_shape=input_shape),
                             Conv2D(3,3,activation='relu',padding='same'),
                             efn.EfficientNetB0(include_top=False,input_cq_transform=(),weights='imagenet'),
                             GlobalAveragePooling2D(),
                             Dense(32,activation='relu'),
                             Dense(1, activation='sigmoid')])

model.summary()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[[keras.metrics.AUC(), 'accuracy']])

16809984/16804768 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 69, 193, 3)        30        
_________________________________________________________________
efficientnet-b0 (Functional) (None, None, None, 1280)  4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                40992     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 4,090,619
Trainable params: 4,048,603
Non-trainable params: 42,016
_________________________________________________________________


In [18]:
model.fit(train_dataset,epochs=1,validation_data=valid_dataset)

2079/2079 [==============================] - 7350s 4s/step - loss: 0.4825 - auc: 0.8230 - val_loss: 0.4447 - val_auc: 0.8523


In [20]:
preds = model.predict(test_dataset)
preds

array([[0.99999154],
       [0.6224144 ],
       [0.36453208],
       ...,
       [0.23474687],
       [0.9827679 ],
       [0.09861477]], dtype=float32)

In [21]:
model.save('./model_efn.h5')

The SOTA model, EfficientNetB0, which we only ran for a single epoch (due to time constraints), we can already see equivalent performance in terms of AUC score and accuracy for the training as well as validation dataset, which is unseen to the model.

In [27]:
# Function to save kaggle submissions for test prediction probabilities
def get_kaggle_format(prediction_probs, model='base'):
    # load the sample submission file
#     sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
    sample_sub['target'] = prediction_probs
    
    # Output filename for kaggle submission
    filename = f"kaggle_sub_{model}.csv"
    
    # Save the DataFrame to a file
    sample_sub.to_csv(filename, index=False)
    print(f'File name: {filename}')

In [28]:
get_kaggle_format(preds, model='efn')

File name: kaggle_sub_efn.csv


We refer to this model in the main modelling notebook.